In [1]:
import json 

In [2]:
with open('raw-news_category-100.psql.json', 'r') as in_file: 
    raw_json = json.load(in_file)

```json
{
    'uuid': '1bfe7f10-e8da-4cde-a8e7-f84c1802a9c7',
    'link': 'https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89',
    'title': 'There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV',
    'description': 'She left her husband. He killed their children. Just another day in America.',
    'date': '2018-05-26',
    'authors': ['Melissa Jeltsen'],
    'category': 'CRIME'
}
```

In [3]:
news_insert_header = """INSERT INTO "news" ("id", "uuid", "link", "title", "description", "created_at", "category") VALUES"""
news_insert_lines = []

authors_insert_header = """INSERT INTO "authors" ("id", "name") VALUES"""
authors_insert_lines = []

news_authors_insert_header = """INSERT INTO "news_authors" ("news_id", "authors_id") VALUES"""
news_authors_insert_lines = []

news_n = 0
authors_name_to_id = {}
authors_n = 0

In [4]:
for j in raw_json: 
    
    news_uuid = j['uuid']
    
    # news_id start with 1
    news_n += 1
    news_id = news_n
    n_insert_line = "({}, '{}', '{}', '{}', '{}', '{}', '{}')".format(
        news_id, 
        news_uuid,
        j['link'],
        j['title'].replace("'", "''"),
        j['description'].replace("'", "''"),
        j['date'],
        j['category'],
    )
    news_insert_lines.append(n_insert_line)
    
    # authors_id
    for a_name in j['authors']: 
        
        if a_name in authors_name_to_id: 
            a_id = authors_name_to_id['a_name']
        else: 
            authors_n += 1
            a_id = authors_n
            authors_name_to_id['a_name'] = a_id
            a_insert_line = "({}, '{}')".format(a_id, a_name.replace("'", "''"))
            authors_insert_lines.append(a_insert_line)
        
        n_a_insert_line = "({}, {})".format(news_id, a_id)
        news_authors_insert_lines.append(n_a_insert_line)
            

In [5]:
news_insert_script = news_insert_header + '\n' + ",\n".join(news_insert_lines) + ";"
authors_insert_script = authors_insert_header + '\n' + ",\n".join(authors_insert_lines) + ";"
news_authors_insert_script = news_authors_insert_header + '\n' + ",\n".join(news_authors_insert_lines) + ";"

with open("insert-news_category.psql.sql", 'w') as out_file: 
    out_file.write(news_insert_script+'\n\n'+authors_insert_script+'\n\n'+news_authors_insert_script)